In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import spacy
import re
import nltk
import string
import sklearn
import tensorflow as tf
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
# !pip install googletrans==4.0.0-rc1

In [ ]:
# !pip install SpeechRecognition pydub

In [ ]:
rd=pd.read_csv("/content/EDA_PP_F.csv",encoding='latin1')
rd.head()

,Unnamed: 0,tweet_text,type,text_len
0,0,in words food crapilicious,none,4.0
1,2,classy whore or red velvet cupcakes,none,6.0
2,3,meh thanks heads up concerned angry dude twitter,none,8.0
3,4,this isis account pretending kurdish account l...,none,9.0
4,5,yes test god good bad indifferent weird whatev...,none,11.0


Baseline Model training

In [ ]:
label_enc = LabelEncoder()
rd['type'] = label_enc.fit_transform(rd['type'])


In [ ]:
X = rd['tweet_text']
y = rd['type']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 6300],
       [   1, 5204],
       [   2, 3927],
       [   3, 6179],
       [   4, 6331],
       [   5, 5843]])

In [ ]:

ros = RandomOverSampler()
X_train, y_train = ros.fit_resample(np.array(X_train).reshape(-1, 1), np.array(y_train).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in X_train], y_train)), columns = ['tweet_text', 'type']);

In [ ]:
X_train = train_os['tweet_text'].values
y_train = train_os['type'].values

In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 6331],
       [   1, 6331],
       [   2, 6331],
       [   3, 6331],
       [   4, 6331],
       [   5, 6331]])

Bag of words

In [ ]:
cv = CountVectorizer()
X_train_cv =  cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

TF-IDF

In [ ]:
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_cv)
X_train_tf = tf_transformer.transform(X_train_cv)
X_test_tf = tf_transformer.transform(X_test_cv)

Model Fitting

In [ ]:
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
ada = AdaBoostClassifier(random_state=42)
lgb = LGBMClassifier(random_state=42)
xgb = XGBClassifier(eval_metric="mlogloss",random_state=42)
dt = DecisionTreeClassifier(random_state=42)
svc = SVC(random_state=42)
nb = MultinomialNB()
mlp = MLPClassifier(random_state=42)

models = {
    "Random Forest": rf,
    "Gradient Boosting":gb,
    "AdaBoost": ada,
    "LightGBM": lgb,
    "XGBoost": xgb,
    "Decision Tree":dt,
    "Support Vector Machine":svc,
    "Naive Bayes": nb,
    "Multilayer Perceptron":mlp
}

In [ ]:
def fit_model(clf,x_train,y_train,x_test, y_test):
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    accuracy = accuracy_score(y_pred, y_test)
    return accuracy

accuracys = []


for name,clf in models.items():
     curr_acc = fit_model(clf,X_train_tf,y_train,X_test_tf,y_test)
     accuracys.append(curr_acc)

In [ ]:
models_df = pd.DataFrame({"Models":models.keys(),"Accuracy Scores":accuracys}).sort_values('Accuracy Scores',ascending=False)
models_df

,Models,Accuracy Scores
0,Random Forest,0.854741


In [ ]:
sentiments = ["age","not bullying","racism","religion","gender"]
all_categories_names=np.array(sentiments)
x=rf.predict(cv.transform(["hi hello"]))
print(all_categories_names[x])

In [ ]:
from googletrans import Translator
translator=Translator()
translated=translator.translate("Alle Muslime sind Terroristen")
print(translated.text)

In [ ]:
x=rf.predict(cv.transform([translated.text]))
print(all_categories_names[x])

Audio Input

In [ ]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [ ]:
from io import BytesIO
from pydub import AudioSegment

def save_audio_file(byte_stream, filename):
    # read the byte stream into an audio segment
    audio_segment = AudioSegment.from_file(BytesIO(byte_stream))

    # set the frame rate of the audio segment
    audio_segment = audio_segment.set_frame_rate(16000)

    # export the audio segment to a file
    audio_segment.export(filename, format="wav")

    print("Audio file saved as:", filename)

In [ ]:
byte_stream = record(5)
save_audio_file(byte_stream, "my_audiofile.wav")

In [ ]:
import speech_recognition as sr
filename = "/content/my_audiofile.wav"
# initialize the recognizer
r = sr.Recognizer()
# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    print(text)

In [ ]:
x=rf.predict(cv.transform([text]))
print(all_categories_names[x])

PyTorch Bi-LSTM RNN

> we will define a custom Bidirectional LSTM using PyTorch in order to perform the Sentiment Analysis on the tweets.





Data preprocessing for LSTM

In [ ]:
#Data preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
#PyTorch LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#Tokenization for LSTM
from collections import Counter
from gensim.models import Word2Vec
#Seed for reproducibility
import random

seed_value=42
import time

from sklearn.metrics import classification_report, confusion_matrix


Train - Test split

In [ ]:
X = rd['tweet_text']
y = rd['type']

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed_value)

Train-Validation split

In [ ]:
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=seed_value)

In [ ]:
# (unique, counts) = np.unique(y_train, return_counts=True)
# np.asarray((unique, counts)).T

Oversampling of training set

In [ ]:
# ros = RandomOverSampler()
# X_train, y_train = ros.fit_resample(np.array(X_train).reshape(-1, 1), np.array(y_train).reshape(-1, 1));
# train_os = pd.DataFrame(list(zip([x[0] for x in X_train], y_train)), columns = ['tweet_text', 'type']);

In [ ]:
# X_train = train_os['tweet_text'].values
# y_train = train_os['type'].values

In [ ]:
# (unique, counts) = np.unique(y_train, return_counts=True)
# np.asarray((unique, counts)).T

In [ ]:
# def Tokenize(column, seq_len):
#     ##Create vocabulary of words from column
#     corpus = [word for text in column for word in text.split()]
#     count_words = Counter(corpus)
#     sorted_words = count_words.most_common()
#     vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

#     ##Tokenize the columns text using the vocabulary
#     text_int = []
#     for text in column:
#         r = [vocab_to_int[word] for word in text.split()]
#         text_int.append(r)
#     ##Add padding to tokens
#     seq_len = int(seq_len)
#     features = np.zeros((len(text_int), seq_len), dtype = int)
#     for i, review in enumerate(text_int):
#         if len(review) <= seq_len:
#             zeros = list(np.zeros(seq_len - len(review)))
#             new = zeros + review
#         else:
#             new = review[: seq_len]
#         features[i, :] = np.array(new)

#     return sorted_words, features

In [ ]:
# max_len = np.max(rd['text_len'])
# max_len 

In [ ]:
# vocabulary, tokenized_column = Tokenize(rd["tweet_text"], max_len)

In [ ]:
# rd["tweet_text"].iloc[10]

In [ ]:
# tokenized_column[10]

In [ ]:
# keys = []
# values = []
# for key, value in vocabulary[:20]:
#     keys.append(key)
#     values.append(value)

In [ ]:
# plt.figure(figsize=(15, 5))
# ax = sns.barplot(x=keys, y=values, palette='mako')
# plt.title('Top 20 most common words', size=25)
# ax.bar_label(ax.containers[0])
# plt.ylabel("Words count")
# plt.show()

In [ ]:
# Word2vec_train_data = list(map(lambda x: x.split(), X_train))

In [ ]:
# EMBEDDING_DIM = 200

In [ ]:
# word2vec_model = Word2Vec(Word2vec_train_data, vector_size=EMBEDDING_DIM)

In [ ]:
# print(f"Vocabulary size: {len(vocabulary) + 1}")

In [ ]:
# VOCAB_SIZE = len(vocabulary) + 1 #+1 for the padding

In [ ]:
# #define empty embedding matrix
# embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
    
# #fill the embedding matrix with the pre trained values from word2vec
# #    corresponding to word (string), token (number associated to the word)
# for word, token in vocabulary:
#     if word2vec_model.wv.__contains__(word):
#         embedding_matrix[token] = word2vec_model.wv.__getitem__(word)

# print("Embedding Matrix Shape:", embedding_matrix.shape)

Train - Validation - Test split

In [ ]:
# X = tokenized_column
# y = rd['type'].values

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed_value)

In [ ]:
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=seed_value)

In [ ]:
# (unique, counts) = np.unique(y_train, return_counts=True)
# np.asarray((unique, counts)).T

In [ ]:
# ros = RandomOverSampler()
# X_train_os, y_train_os = ros.fit_resample(np.array(X_train),np.array(y_train));

In [ ]:
# (unique, counts) = np.unique(y_train_os, return_counts=True)
# np.asarray((unique, counts)).T

PyTorch datasets and dataloaders

In [ ]:
# import torch
# from torch.utils.data import TensorDataset
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train_encoded = le.fit_transform(y_train_os)
# y_test_encoded = le.fit_transform(y_test)
# y_valid_encoded = le.fit_transform(y_valid)
# # Convert NumPy arrays to PyTorch tensors
# Xtr_tensor = torch.tensor(X_train_os, dtype=torch.int64)
# Ytr_tensor = torch.tensor(y_train_encoded, dtype=torch.int64)
# Xtt_tensor = torch.tensor(X_test, dtype=torch.int64)
# Ytt_tensor = torch.tensor(y_test_encoded, dtype=torch.int64)
# Xv_tensor = torch.tensor(X_valid, dtype=torch.int64)
# Yv_tensor = torch.tensor(y_valid_encoded, dtype=torch.int64)
# # Create a TensorDataset from tensors
# dataset_Xtr = TensorDataset(Xtr_tensor,Ytr_tensor)
# dataset_Xtt = TensorDataset(Xtt_tensor,Ytt_tensor)
# dataset_Xv = TensorDataset(Xv_tensor,Yv_tensor)

In [ ]:
# from torch.utils.data import DataLoader

# # Create a DataLoader from the TensorDataset
# BATCH_SIZE = 32
# dataloader_Xtr = DataLoader(dataset_Xtr, batch_size=BATCH_SIZE, shuffle=True)
# dataloader_Xtt = DataLoader(dataset_Xtt, batch_size=BATCH_SIZE, shuffle=True)
# dataloader_Xv = DataLoader(dataset_Xv, batch_size=BATCH_SIZE, shuffle=True)

Pytorch LSTM Modelling

In [ ]:
# NUM_CLASSES = 5 #We are dealing with a multiclass classification of 5 classes
# HIDDEN_DIM = 100 #number of neurons of the internal state (internal neural network in the LSTM)
# LSTM_LAYERS = 1 #Number of stacked LSTM layers
# BATCH_SIZE=32
# LR = 3e-4 #Learning rate
# DROPOUT = 0.5 #LSTM Dropout
# BIDIRECTIONAL = True #Boolean value to choose if to use a bidirectional LSTM or not
# EPOCHS = 5 #Number of training epoch

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# class BiLSTM_Sentiment_Classifier(nn.Module):

#     def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, lstm_layers, bidirectional,batch_size, dropout):
#         super(BiLSTM_Sentiment_Classifier,self).__init__()
        
#         self.lstm_layers = lstm_layers
#         self.num_directions = 2 if bidirectional else 1
#         self.hidden_dim = hidden_dim
#         self.num_classes = num_classes
#         self.batch_size = batch_size
        

#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
#         self.lstm = nn.LSTM(embedding_dim,
#                             hidden_dim,
#                             num_layers=lstm_layers,
#                             dropout=dropout,
#                             bidirectional=bidirectional,
#                             batch_first=True)

#         self.fc = nn.Linear(hidden_dim*self.num_directions, num_classes)
#         self.softmax = nn.LogSoftmax(dim=1)
        
#     def forward(self, x, hidden):
#         self.batch_size = x.size(0)
#         ##EMBEDDING LAYER
#         embedded = self.embedding(x)
#         #LSTM LAYERS
#         out, hidden = self.lstm(embedded, hidden)
#         #Extract only the hidden state from the last LSTM cell
#         out = out[:,-1,:]
#         #FULLY CONNECTED LAYERS
#         out = self.fc(out)
#         out = self.softmax(out)

#         return out, hidden

#     def init_hidden(self, batch_size):
#         #Initialization of the LSTM hidden and cell states
#         h0 = torch.zeros((self.lstm_layers*self.num_directions, batch_size, self.hidden_dim)).detach().to(DEVICE)
#         c0 = torch.zeros((self.lstm_layers*self.num_directions, batch_size, self.hidden_dim)).detach().to(DEVICE)
#         hidden = (h0, c0)
#         return hidden

In [ ]:
# model = BiLSTM_Sentiment_Classifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,NUM_CLASSES, LSTM_LAYERS,BIDIRECTIONAL, BATCH_SIZE, DROPOUT)
# model = model.to(DEVICE)

# #Initialize embedding with the previously defined embedding matrix
# model.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
# #Allow the embedding matrix to be fined tuned to better adapt to out dataset and get higher accuracy
# model.embedding.weight.requires_grad=True

# print(model)

In [ ]:
# criterion = nn.NLLLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay = 5e-6)

LSTM Training loop

In [ ]:
# total_step = len(dataloader_Xtr)
# total_step_val = len(dataloader_Xv)

# early_stopping_patience = 4
# early_stopping_counter = 0

# valid_acc_max = 0 # Initialize best accuracy top 0

# for e in range(EPOCHS):

#     #lists to host the train and validation losses of every batch for each epoch
#     train_loss, valid_loss  = [], []
#     #lists to host the train and validation accuracy of every batch for each epoch
#     train_acc, valid_acc  = [], []

#     #lists to host the train and validation predictions of every batch for each epoch
#     y_train_list, y_val_list = [], []

#     #initalize number of total and correctly classified texts during training and validation
#     correct, correct_val = 0, 0
#     total, total_val = 0, 0
#     running_loss, running_loss_val = 0, 0


#     ####TRAINING LOOP####

#     model.train()

#     for inputs, labels in dataloader_Xtr:
#         inputs, labels = inputs.to(DEVICE), labels.to(DEVICE) #load features and targets in device

#         h = model.init_hidden(labels.size(0))

#         model.zero_grad() #reset gradients 

#         output, h = model(inputs,h) #get output and hidden states from LSTM network
        
#         loss = criterion(output, labels)
#         loss.backward()
        
#         running_loss += loss.item()
        
#         optimizer.step()

#         y_pred_train = torch.argmax(output, dim=1) #get tensor of predicted values on the training set
#         y_train_list.extend(y_pred_train.squeeze().tolist()) #transform tensor to list and the values to the list
        
#         correct += torch.sum(y_pred_train==labels).item() #count correctly classified texts per batch
#         total += labels.size(0) #count total texts per batch

#     train_loss.append(running_loss / total_step)
#     train_acc.append(100 * correct / total)

#     ####VALIDATION LOOP####
    
#     with torch.no_grad():
        
#         model.eval()
        
#         for inputs, labels in dataloader_Xv:
#             inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

#             val_h = model.init_hidden(labels.size(0))

#             output, val_h = model(inputs, val_h)

#             val_loss = criterion(output, labels)
#             running_loss_val += val_loss.item()

#             y_pred_val = torch.argmax(output, dim=1)
#             y_val_list.extend(y_pred_val.squeeze().tolist())

#             correct_val += torch.sum(y_pred_val==labels).item()
#             total_val += labels.size(0)

#         valid_loss.append(running_loss_val / total_step_val)
#         valid_acc.append(100 * correct_val / total_val)

#     #Save model if validation accuracy increases
#     if np.mean(valid_acc) >= valid_acc_max:
#         torch.save(model.state_dict(), './state_dict.pt')
#         print(f'Epoch {e+1}:Validation accuracy increased ({valid_acc_max:.6f} --> {np.mean(valid_acc):.6f}).  Saving model ...')
#         valid_acc_max = np.mean(valid_acc)
#         early_stopping_counter=0 #reset counter if validation accuracy increases
#     else:
#         print(f'Epoch {e+1}:Validation accuracy did not increase')
#         early_stopping_counter+=1 #increase counter if validation accuracy does not increase
        
#     if early_stopping_counter > early_stopping_patience:
#         print('Early stopped at epoch :', e+1)
#         break
    
#     print(f'\tTrain_loss : {np.mean(train_loss):.4f} Val_loss : {np.mean(valid_loss):.4f}')
#     print(f'\tTrain_acc : {np.mean(train_acc):.3f}% Val_acc : {np.mean(valid_acc):.3f}%')

In [ ]:
# # Loading the best model
# model.load_state_dict(torch.load('./state_dict.pt'))

LSTM Testing

In [ ]:
# model.eval()
# y_pred_list = []
# y_test_list = []
# for inputs, labels in dataloader_Xtt:
#     inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
#     test_h = model.init_hidden(labels.size(0))

#     output, val_h = model(inputs, test_h)
#     y_pred_test = torch.argmax(output, dim=1)
#     y_pred_list.extend(y_pred_test.squeeze().tolist())
#     y_test_list.extend(labels.squeeze().tolist())

In [ ]:
# def conf_matrix(y, y_pred, title, labels):
#     fig, ax =plt.subplots(figsize=(15,15))
#     ax=sns.heatmap(confusion_matrix(y, y_pred), annot=True, cmap="Purples", fmt='g', cbar=False, annot_kws={"size":30})
#     plt.title(title, fontsize=25)
#     ax.xaxis.set_ticklabels(labels, fontsize=16) 
#     ax.yaxis.set_ticklabels(labels, fontsize=14.5)
#     ax.set_ylabel('Test', fontsize=25)
#     ax.set_xlabel('Predicted', fontsize=25)
#     plt.show()

In [ ]:

# conf_matrix(y_test_list, y_pred_list, 'PyTorch Bi-LSTM Sentiment Analysis\nConfusion Matrix', ['NoTCyberbullying', 'Sexism', 'Religion', 'Age', 'Racism'])

In [ ]:
# print('Classification Report for Bi-LSTM:\n', classification_report(y_test_list, y_pred_list, target_names=['NoTCyberbullying', 'Sexism', 'Religion', 'Age', 'Racism']))


BERT 

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Transformers library for BERT
import transformers
from transformers import BertModel
from transformers import BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

from sklearn.metrics import classification_report, confusion_matrix

#Seed for reproducibility
import random

seed_value=42
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
X = rd['tweet_text'].values
y = rd['type'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed_value)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=seed_value)

In [ ]:
ros = RandomOverSampler()
X_train_os, y_train_os = ros.fit_resample(np.array(X_train).reshape(-1,1),np.array(y_train).reshape(-1,1))

In [ ]:
X_train_os = X_train_os.flatten()
y_train_os = y_train_os.flatten()

In [ ]:
(unique, counts) = np.unique(y_train_os, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 5698],
       [   1, 5698],
       [   2, 5698],
       [   3, 5698],
       [   4, 5698]])

BERT Tokenization

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
def bert_tokenizer(data):
    input_ids = []
    attention_masks = []
    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text=sent,
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]` special tokens
            max_length=MAX_LEN,             # Choose max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length 
            return_attention_mask=True      # Return attention mask
            )
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
# Tokenize train tweets
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in X_train]

# Find the longest tokenized tweet
max_len = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len)

Max length:  77


In [ ]:
MAX_LEN = 80

In [ ]:
train_inputs, train_masks = bert_tokenizer(X_train_os)
val_inputs, val_masks = bert_tokenizer(X_valid)
test_inputs, test_masks = bert_tokenizer(X_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Convert target columns to pytorch tensors format
train_labels = torch.from_numpy(y_train_os)
val_labels = torch.from_numpy(y_valid)
test_labels = torch.from_numpy(y_test)

Dataloaders

In [ ]:
batch_size = 32

In [ ]:
# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

BERT Modeling

In [ ]:
class Bert_Classifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(Bert_Classifier, self).__init__()
        # Specify hidden size of BERT, hidden size of the classifier, and number of labels
        n_input = 768
        n_hidden = 50
        n_output = 5
        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Add dense layers to perform the classification
        self.classifier = nn.Sequential(
            nn.Linear(n_input,  n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_output)
        )
        # Add possibility to freeze the BERT model
        # to avoid fine tuning BERT params (usually leads to worse results)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        # Feed input data to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [ ]:
def initialize_model(epochs=4):
    # Instantiate Bert Classifier
    bert_classifier = Bert_Classifier(freeze_bert=False)
    
    bert_classifier.to(device)

    # Set up optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # learning rate, set to default value
                      eps=1e-8    # decay, set to default value
                      )
    
    ### Set up learning rate scheduler ###

    # Calculate total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Defint the scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
EPOCHS=1

In [ ]:
bert_classifier, optimizer, scheduler = initialize_model(epochs=EPOCHS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

BERT Training

In [ ]:
# Define Cross entropy Loss function for the multiclass classification task
loss_fn = nn.CrossEntropyLoss()

def bert_train(model, train_dataloader, val_dataloader=None, epochs=1, evaluation=False):

    print("Start training...\n")
    for epoch_i in range(epochs):
        print("-"*10)
        print("Epoch : {}".format(epoch_i+1))
        print("-"*10)
        print("-"*38)
        print(f"{'BATCH NO.':^7} | {'TRAIN LOSS':^12} | {'ELAPSED (s)':^9}")
        print("-"*38)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0
        
        ###TRAINING###

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass and get logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update model parameters:
            # fine tune BERT params and train additional dense layers
            optimizer.step()
            # update learning rate
            scheduler.step()

            # Print the loss values and time elapsed for every 100 batches
            if (step % 100 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch
                
                print(f"{step:^9} | {batch_loss / batch_counts:^12.6f} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        ###EVALUATION###
        
        # Put the model into the evaluation mode
        model.eval()
        
        # Define empty lists to host accuracy and validation for each batch
        val_accuracy = []
        val_loss = []

        for batch in val_dataloader:
            batch_input_ids, batch_attention_mask, batch_labels = tuple(t.to(device) for t in batch)
            
            # We do not want to update the params during the evaluation,
            # So we specify that we dont want to compute the gradients of the tensors
            # by calling the torch.no_grad() method
            with torch.no_grad():
                logits = model(batch_input_ids, batch_attention_mask)

            loss = loss_fn(logits, batch_labels)

            val_loss.append(loss.item())

            # Get the predictions starting from the logits (get index of highest logit)
            preds = torch.argmax(logits, dim=1).flatten()

            # Calculate the validation accuracy 
            accuracy = (preds == batch_labels).cpu().numpy().mean() * 100
            val_accuracy.append(accuracy)

        # Compute the average accuracy and loss over the validation set
        val_loss = np.mean(val_loss)
        val_accuracy = np.mean(val_accuracy)
        
        # Print performance over the entire training data
        time_elapsed = time.time() - t0_epoch
        print("-"*61)
        print(f"{'AVG TRAIN LOSS':^12} | {'VAL LOSS':^10} | {'VAL ACCURACY (%)':^9} | {'ELAPSED (s)':^9}")
        print("-"*61)
        print(f"{avg_train_loss:^14.6f} | {val_loss:^10.6f} | {val_accuracy:^17.2f} | {time_elapsed:^9.2f}")
        print("-"*61)
        print("\n")
    
    print("Training complete!")

In [ ]:
bert_train(bert_classifier, train_dataloader, val_dataloader, epochs=EPOCHS)

Start training...

----------
Epoch : 1
----------
--------------------------------------
BATCH NO. |  TRAIN LOSS  | ELAPSED (s)
--------------------------------------
   100    |   0.661241   |  2314.06 
   200    |   0.299919   |  2212.20 
   300    |   0.256707   |  2218.87 
   400    |   0.228211   |  2218.81 
   500    |   0.220976   |  2207.50 
   600    |   0.197645   |  2194.91 
   700    |   0.211520   |  2206.83 
   800    |   0.188363   |  2199.19 
   890    |   0.180124   |  1995.62 
-------------------------------------------------------------
AVG TRAIN LOSS |  VAL LOSS  | VAL ACCURACY (%) | ELAPSED (s)
-------------------------------------------------------------
   0.273098    |  0.196382  |       93.64       | 20436.96 
-------------------------------------------------------------


Training complete!


In [ ]:
import pickle
pickle.dump(mnb,open('model.pkl','wb'))

NameError: ignored

In [ ]:
def bert_predict(model, test_dataloader):
    
    # Define empty list to host the predictions
    preds_list = []
    
    # Text input to classify
    text = "This is a hate speech comment"

    # Tokenize text input
    input_ids, attention_masks = bert_tokenizer([text])

    # Put the model into evaluation mode
    model.eval()
    
    for batch in test_dataloader:
        batch_input_ids, batch_attention_mask = tuple(t.to(device) for t in batch)[:2]
        
        # Avoid gradient calculation of tensors by using "no_grad()" method
        with torch.no_grad():
            logit = model(batch_input_ids, batch_attention_mask)
        
        # Get index of highest logit
        pred = torch.argmax(logit,dim=1).cpu().numpy()
        # Append predicted class to list
        preds_list.extend(pred)

    return preds_list
    # Print predicted class 
    if pred == 0:
      print("The text belongs to class 0: non-hate speech")
    else:
      print("The text belongs to class 1: hate speech")